## Group 12 _ Assignment 4

In [39]:
# Setting up the root directory
import os
root_dir =r'C:\Users\zeeni\OneDrive\Documents\NEU\Sem 2\DSEM\Assignments\Assignment4_ZeeniaSingla_Team12\Assignment4'

### Reading Data from data.xlsx
we are using pandas for doing so

In [40]:
import pandas as pd

os.chdir(root_dir)
dataset = "data.xlsx"
data = pd.read_excel(dataset)
data.head() # reading data from excel sheel

,Image,Language 1,Language 2
0,1.JPEG,Is all I want to\nfoster in real \nworld while...,क्या मैं अपने विचारों\nको पथपाकर वास्तविक \nदु...
1,2.JPEG,Do I know \nwhat I want \nfrom life or \nam I ...,क्या मुझे पता है कि \nमुझे जीवन से क्या \nचाहि...
2,3.JPEG,Flying amidst the\nclouds in the \nmiddle of n...,"कहीं भी बीच में \nबादलों के बीच \nउड़ते हुए, म..."
3,4.JPEG,Boston: \nWhile embarking on this \nnew chapte...,बोस्टन:\nजीवन के इस \nनए अध्याय \nकी शुरुआत \n...
4,5.JPEG,\nLooking at the moon \ningites my \nthoughtf...,चाँद को देखकर मेरी\nविचारशील राज्य।\nमुझे इसमे...


### Cartoonization of the images
we are running a loop with images from the root_dir and saving them in new folder ("cartoons")

In [41]:
from PIL import Image, ImageFont, ImageDraw, ImageOps
import cv2 
from IPython.display import display
from PIL import Image as pili, ImageDraw as pild, ImageFont as pilf 

os.chdir(root_dir)

# with the OpenCV function imread(), the order of colors is BGR (blue, green, red)
for myImage in data.Image:
    img = cv2.imread(myImage)

    # In Pillow, the order of colors is assumed to be RGB (red, green, blue).
    # As we are using Image.fromarray() of PIL module, we need to convert BGR to RGB.
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # Converting BGR to RGB

    def imgcompress_mem(path_in, k):
        img = cv2.imread(path_in, cv2.IMREAD_UNCHANGED)

        # set the ratio of resized image
        width = int((img.shape[1])/k)
        height = int((img.shape[0])/k)
        #width = 400
        #height = 400

        # resize the image by resize() function of openCV library
        return cv2.resize(img, (width, height), interpolation=cv2.INTER_AREA)

    def cartoonizebl_mem(path_in, k, blur, line):

        imgc = imgcompress_mem(path_in, k)
        #imgc_pil = cv2.cvtColor(imgc, cv2.COLOR_BGR2RGB) # Converting BGR to RGB
        #display(Image.fromarray(imgc_pil))

        line_size = line
        blur_value = blur
        #imgc = cv2.imread(path_out, cv2.IMREAD_UNCHANGED)
        gray = cv2.cvtColor(imgc, cv2.COLOR_BGR2GRAY)
        gray_blur = cv2.medianBlur(gray, blur_value)
        bigedges = cv2.adaptiveThreshold(gray_blur, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, line_size, blur_value)
        bigedges_pil = cv2.cvtColor(bigedges, cv2.COLOR_BGR2RGB) # Converting BGR to RGB
        #display(Image.fromarray(bigedges_pil))

        return cv2.bitwise_and(imgc, imgc, mask=bigedges)

    cblimg = cartoonizebl_mem(myImage, 1, 5, 13)
    # cblimg_pil = cv2.cvtColor(cblimg, cv2.COLOR_BGR2RGB) # Converting BGR to RGB
    img = cblimg

    final_path = os.path.join(root_dir, 'cartoons')
    if not os.path.exists(final_path):
        os.mkdir(final_path)
    cv2.imwrite(os.path.join(final_path, myImage),img)
print("Done")

Done


### Adding Borders to the images
we are re-writing the images in cartoons folder with white borders

In [42]:
# Adding borders
from PIL import Image as pili, ImageOps as piliops

current_path = os.path.join(root_dir, "cartoons")
if os.path.exists(current_path):
    os.chdir(current_path) # setting current working directory
    for myImage in data.Image:
        piliops.expand(pili.open(myImage), border=(15,15),fill='white').save(os.path.join(current_path,myImage))
    print("Done")
else:
    print("Run previous block first!!")

Done


### Adding subtitles / Dialogs
##### Input for the Language option is required for generating with particular language
Default will generate with English subtitles
The newly generated images will be saved in new folder ("cartoon_sub")

In [43]:
from PIL import Image as pili, ImageDraw as pild, ImageFont as pilf
from IPython.display import display
import textwrap

# print(os.getcwd())
font_path = os.path.join(root_dir, "language1.ttf")
lan_select = input("Select a language: press 1 for language1 and 2 for language2 ")
if int(lan_select) == 2:
    font_path = os.path.join(root_dir, "language2.ttf")
subtitles_images_path = os.path.join(root_dir, "cartoon_sub")

TINT_COLOR = (0, 0, 0)  # Black
TRANSPARENCY = .45  # Degree of transparency, 0-100%
OPACITY = int(255 * TRANSPARENCY)

from PIL import Image, ImageFont, ImageDraw

for a in range(len(data.Image)):
    img = pili.open(data.Image[a]).convert('RGBA')
    
    overlay = pili.new('RGBA', img.size, TINT_COLOR+(0,))
    draw = pild.Draw(overlay)
    font = pilf.truetype(font_path,40)
    #x, y = (img.width - 510, img.height-100)
    text = data["Language 1"][a]
    if int(lan_select) == 2:
        text = data["Language 2"][a]
    w, h = font.getsize(text)
    num_lines = len(text.split('\n'))
    x, y = 15, img.height - 15 - (num_lines-0.1*num_lines)*h
#     text = textwrap.fill(text=text, width=50)
    #draw.rectangle((x, y, x + w, y + h), fill='black')
    #draw.rectangle((x, y, x + w, y + 4*h), fill=TINT_COLOR+(OPACITY,))
    draw.rectangle((x, y, x + img.width - 30, y + (num_lines-0.1*num_lines)*h), fill=TINT_COLOR+(OPACITY,))
    draw.text((x, y), text, fill=(209, 239, 8), font=font)
    # Alpha composite these two images together to obtain the desired result.
    img = pili.alpha_composite(img, overlay)
    img = img.convert("RGB") # Remove alpha for saving in jpg format.
    if not os.path.exists(subtitles_images_path):
        os.mkdir(subtitles_images_path)
    img.save(os.path.join(subtitles_images_path, data.Image[a]))
print("Done")

Select a language: press 1 for language1 and 2 for language2 2
Done


### Horizontal Image Stacking
##### Input for operating system is required for error free working (1 for mac and 2 for windows)
Random number of images between 2 to 5 are stacked together to create a horizontal image
These images will be saved in a new folder ("horizontal_stack")

Before generating these files, we are making sure to remove that kind of folder if it exists and recreate it

In [44]:
import numpy as np
import PIL
from IPython.display import Image
from IPython.display import display
from PIL import Image as pili, ImageOps as piliops
import shutil

horizontal_stack_path = os.path.join(root_dir,"horizontal_stack")

if os.path.exists(horizontal_stack_path):
    shutil.rmtree(horizontal_stack_path, ignore_errors=True)

mac_or_pc = input("Are you using Mac ? Press 1 for yes and 2 for no : ")


def gridImage(imgList, d_directory):
    dim = imgList[0]
    if int(mac_or_pc) == 2:
        dim = imgList[0].split("\\")
    else :
        dim = imgList[0].split("/")
    imgNumber = dim[len(dim)-1]
    imgs    = [ PIL.Image.open(i) for i in imgList ]
    # pick the image which is the smallest, and resize the others to match it (can be arbitrary image shape here)
    
    min_shape = sorted( [(np.sum(i.size), i.size ) for i in imgs])[0][1]
#     print("Array size "+np.asarray( i.resize(min_shape) ) for i in imgs)
    imgs_comb = np.hstack( [np.asarray( i.resize(min_shape) ) for i in imgs ] )

    # save that beautiful picture
    imgs_comb = PIL.Image.fromarray(imgs_comb)
    if not os.path.exists(horizontal_stack_path):
        os.mkdir(horizontal_stack_path)
    imgs_comb.save( os.path.join(horizontal_stack_path, imgNumber) )
    
# To get the random number of images per line (3 to 5) 
os.chdir(subtitles_images_path)
# os.getcwd()
list_im = [ (os.path.join(subtitles_images_path, file)) for file in data.Image]
while 0 < len(list_im):
    lines = 0
#     print("new page!")
    num_cols = old_num_cols = 0
    while lines < 4 and 0 < len(list_im):
        lines += 1
        while num_cols == old_num_cols:
            num_cols = np.random.randint(2, 5)
        old_num_cols = num_cols
        window = list_im[:num_cols]
#         print(window)
        gridImage(window, subtitles_images_path)
#         print((print (i) for i in window))
        list_im = list_im[num_cols:]
print("Done")

Are you using Mac ? Press 1 for yes and 2 for no : 2
Done


### Vertical Image stacking
We are trying to stack atleast five horizontally stacked pictures to a single vertical stack
These files will be saved in new folder(vertical_stack)

In [45]:
# os.getcwd()
import shutil
os.chdir(horizontal_stack_path)
import re
def tryint(s):
    try:
        return int(s)
    except:
        return s
    
def alphanum_key(s):
    """ Turn a string into a list of string and number chunks.
        "z23a" -> ["z", 23, "a"]
    """
    return [ tryint(c) for c in re.split('([0-9]+)', s) ]

def sort_images(l):
    """ Sort the given list in the way that humans expect.
    """
    l.sort(key=alphanum_key)
    return l

vertical_stack_path = os.path.join(root_dir,"vertical_stack")

if os.path.exists(vertical_stack_path):
    shutil.rmtree(vertical_stack_path, ignore_errors=True)

horimages = []
for i in os.listdir(horizontal_stack_path):
        horimages.append(i)
horimages = sort_images(horimages)
# print(horimages)
        
list1 = horimages[0:5]
list2 = horimages[5:10]
list3 = horimages[10:15]
list4 = horimages[15:20]
list5 = horimages[20:25]
list6 = horimages[25:30]

def verticalImage(imgList, directory):
#     print(imgList)
    imgNumber = imgList[0]
    imgs    = [ PIL.Image.open(i) for i in imgList ]

    # pick the image which is the smallest, and resize the others to match it (can be arbitrary image shape here)
    min_shape = sorted( [(np.sum(i.size), i.size ) for i in imgs])[0][1]
    imgs_comb = np.vstack( [np.asarray( i.resize(min_shape) ) for i in imgs ] )

    # save that beautiful picture
    imgs_comb = PIL.Image.fromarray(imgs_comb)
    imgs_comb.save(os.path.join(vertical_stack_path, imgNumber))
    
if not os.path.exists(vertical_stack_path):
    os.mkdir(vertical_stack_path)
if len(list1) > 0:
    verticalImage(list1, horizontal_stack_path)
if len(list2) > 0:
    verticalImage(list2, horizontal_stack_path)
if len(list3) > 0:
    verticalImage(list3, horizontal_stack_path)
if len(list4) > 0:
    verticalImage(list4, horizontal_stack_path)
if len(list5) > 0:
    verticalImage(list5, horizontal_stack_path)
if len(list6) > 0:
    verticalImage(list6, horizontal_stack_path)
print("Done")

Done


### Finally the PDF
we are creating pdf file with name "MY_COMIC.PDF" and storing them in PDF folder from the existing vertical_stack images

In [46]:
os.chdir(vertical_stack_path)
pdf_path = os.path.join(root_dir,"pdf")
from PIL import Image
from fpdf import FPDF

#pdf = FPDF()
cover = Image.open(os.path.join(vertical_stack_path,"1.JPEG")) # hardcoded
width, height = cover.size
pdf = FPDF(unit = "pt", format = [width, height])
    
# imagelist is the list with all image filenames
imagelist = sort_images(os.listdir(vertical_stack_path)) # sorted image list
# print(imagelist)

for image in imagelist:
    pdf.add_page()
    pdf.image(image, 0, 0, width, height)
    if not os.path.exists(pdf_path):
        os.mkdir(pdf_path)
pdf.output(os.path.join(pdf_path, "MY_COMIC.PDF"), "F")
print("done! PDF file is saved in pdf folder in the root directory")

done! PDF file is saved in pdf folder in the root directory


## THANK YOU!!